In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Workload-Nutzung

<span id="usage"></span>
Nutzung ist ein Maß für die Zeitspanne, in der die QPU für Ihre Workload gesperrt ist, und sie wird unterschiedlich berechnet, je nachdem, welchen Ausführungsmodus Sie verwenden.

* Session-Nutzung ist die Wanduhr-Zeit, in der die Session aktiv ist. Weitere Informationen zum Session-Status-Übergang finden Sie unter [Session length](/guides/run-jobs-session#session-length).
* Batch-Nutzung ist die Summe der Quantenzeit (Zeit, die der QPU-Komplex zur Verarbeitung Ihres Jobs benötigt) aller Jobs im Batch.
* Einzeljob-Nutzung ist die Quantenzeit, die der Job bei der Verarbeitung verwendet.

Beachten Sie, dass fehlgeschlagene oder abgebrochene Jobs unter bestimmten Umständen zu Ihrer Nutzung zählen - siehe den Abschnitt [Failed and canceled jobs](#failed-job) für Details.

Für Nutzer mit kostenpflichtigem Plan bestimmt die Nutzung, wie viel die Workload kostet. Weitere Details finden Sie unter [Manage cost](/guides/manage-cost).

<span id="failed-job"></span>
## Nutzung für fehlgeschlagene und abgebrochene Jobs
Wenn ein Job fehlschlägt oder abgebrochen wird, wird die gemeldete Nutzung wie folgt berechnet:

* Job- oder Batch-Modus: Die gemeldete Nutzung ist die Zeit, in der die QPU für die Ausführung Ihrer Workload gesperrt war, bis zu dem Zeitpunkt, an dem sie fehlschlug oder abgebrochen wurde. Wenn der Fehler oder Abbruch vor der Sperrung auftrat, beträgt die gemeldete Nutzung daher null. Andernfalls ist die gemeldete Nutzung der Workload die Nutzungsmenge vor dem Fehlschlagen oder Abbrechen der Workload. Daher erscheinen einige fehlgeschlagene Jobs nicht in Ihrer gemeldeten Nutzung und andere schon.

* Session-Modus: Die gemeldete Nutzung ist die Wanduhr-Zeit, in der die Session aktiv ist, unabhängig von der Anzahl der fehlgeschlagenen oder abgebrochenen Jobs.

<span id="view-usage"></span>
## Tatsächliche Nutzung einer Workload abfragen
Nachdem eine Workload abgeschlossen wurde, gibt es mehrere Möglichkeiten, ihre tatsächliche Nutzung anzuzeigen:

- Führen Sie [`batch.usage()`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/batch#usage) oder [`session.usage()`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session#usage) in `qiskit-ibm-runtime` 0.30 oder später aus. Wenn Sie eine ältere Version von `qiskit-ibm-runtime` (>= 0.23 und < 0.30) verwenden, kann die Nutzung immer noch in `session.details()["usage_time"]` und `batch.details()["usage_time"]` gefunden werden.
- Verwenden Sie [`GET /sessions/{id}`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/tags/sessions#tags__sessions__operations__GetSessionDetailsExtendedController_getSessionDetails), um die Nutzung für einen bestimmten Batch oder eine Session anzuzeigen.
- Verwenden Sie [`GET /jobs/{id}`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/tags/jobs#tags__jobs__operations__GetJobByIdController_getJobById), um die Nutzung für einen einzelnen Job anzuzeigen.

<span id="instance-usage"></span>
## Instanz-Nutzung anzeigen
Sie können die Nutzung einer Instanz auf der Seite [Instances](https://quantum.cloud.ibm.com/instances) oder für Personen mit entsprechender Berechtigung auf der Seite [Analytics](https://quantum.cloud.ibm.com/analytics) anzeigen. Beachten Sie, dass die Seiten unterschiedliche Nutzungszahlen anzeigen können, da sie die Nutzung unterschiedlich berechnen.

Die Seite Instances zeigt die Echtzeit-Nutzung der letzten 28 Tage (rollierend) bis zur aktuellen Zeit am aktuellen Tag. Die Nutzung auf der Seite Analytics wird stündlich neu berechnet und umfasst die letzten 28 vollständigen Tage; das heißt, sie zeigt die Nutzung von 00:00 Uhr vor 28 Tagen bis heute zur vollen Stunde.

## Nutzung vor dem Einreichen eines Jobs schätzen
Während eine genaue lokale Schätzung durch die zusätzlichen Operationen für Fehlerunterdrückung und -korrektur erschwert wird, können Sie diese Basisformel verwenden, um eine Annäherung der geschätzten Nutzung zu erhalten:

`<per Sub-Job-Overhead> + (rep_delay + <circuit length>) * <num executions>`

- `<per Sub-Job-Overhead>` ist ein Overhead von ungefähr 2s pro Sub-Job. Dies umfasst Operationen wie das Laden der Payload in die Kontrollelektronik. Ihr Primitiv-Job kann in mehrere Sub-Jobs aufgeteilt werden, wenn er zu groß ist, damit die Ausführungs-Engine ihn auf einmal verarbeiten kann.
- `rep_delay` ist eine [vom Benutzer anpassbare](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-execution-options-v2#rep_delay) Option, und der Standardwert wird durch `backend.default_rep_delay` angegeben, das auf den meisten IBM Quantum-Backends 250 Mikrosekunden beträgt. Beachten Sie, dass das Senken von `rep_delay` die Gesamt-QPU-Ausführungszeit verringert, jedoch auf Kosten einer erhöhten Zustandsvorbereitungsfehlerrate; siehe den [Dynamic repetition rate execution](/guides/repetition-rate-execution)-Guide für weitere Informationen.
- `<circuit length>` ist die Gesamtlänge der Anweisungen. Jede Anweisung benötigt unterschiedlich viel Zeit auf der QPU, daher variiert die Gesamtlänge von Schaltung zu Schaltung. Eine Messung kann beispielsweise 56 Mal länger dauern als ein `x`-Gate. `backend.target[<instruction>][<qubit>].duration` kann verwendet werden, um die genaue Dauer für jede Anweisung zu finden. Eine typische Schaltungslänge liegt wahrscheinlich zwischen 50-100 Mikrosekunden. Wenn Sie Fehlerunterdrückungs- oder -korrektur-Techniken mit den Primitiven verwenden, werden möglicherweise zusätzliche Anweisungen in Ihre Schaltung eingefügt, was die Gesamtschaltungslänge erhöht.
    > **Note:** Die [experimentelle `scheduler_timing`-Option](/guides/visualize-circuit-timing) gibt die Gesamtschaltungszeit zurück, aber dies ist NICHT die für die Abrechnung verwendete Zeit.
- `<num executions>` ist die Gesamtzahl der Schaltungen mal die Anzahl der Shots, wobei die Schaltungen diejenigen sind, die nach dem Broadcasting der PUB-Elemente generiert werden.
    - Wenn Sie Fehlerkorrektur-Techniken mit den Primitiven verwenden, werden möglicherweise zusätzliche Schaltungen als Teil des Korrekturprozesses ausgeführt, was die Gesamtzahl der Ausführungen erhöht. Darüber hinaus kommen fortgeschrittene Fehlerkorrektur-Techniken wie PEA und PEC mit viel höherem Overhead, da sie Schaltungen für das Noise Learning ausführen müssen.
    - Estimator gruppiert qubit-weise kommutierende Observablen, was die Anzahl der Ausführungen reduziert.

Wenn Sie keine fortgeschrittenen Fehlerkorrektur-Techniken oder benutzerdefinierte `rep_delay` verwenden, können Sie `2+0.00035*<num executions>` als schnelle Formel verwenden.

## Nächste Schritte
> **Tip:** - Überprüfen Sie diese Tipps: [Minimize job run time](minimize-time).
>     - Legen Sie die [Maximum execution time](max-execution-time) fest.
>     - Erfahren Sie, wie Sie lokal transpilieren, im Abschnitt [Transpile](./transpile/).
>     - Probieren Sie den Guide [Compare transpiler settings](/guides/circuit-transpilation-settings) aus.